In [29]:
from __future__ import print_function
from keras.models import Model
from keras.layers import Input, LSTM, Dense, Dropout
import numpy as np
from keras.utils.vis_utils import plot_model

%matplotlib inline

from tensorflow.python import keras
from matplotlib import pyplot as plt
from IPython.display import clear_output
import tensorflow as tf
from tensorflow.python.keras.backend import set_session
'''Sequence to sequence example in Keras (character-level).
This script demonstrates how to implement a basic character-level
sequence-to-sequence model. We apply it to translating
short English sentences into short French sentences,
character-by-character. Note that it is fairly unusual to
do character-level machine translation, as word-level
models are more common in this domain.
# Summary of the algorithm
- We start with input sequences from a domain (e.g. English sentences)
    and corresponding target sequences from another domain
    (e.g. French sentences).
- An encoder LSTM turns input sequences to 2 state vectors
    (we keep the last LSTM state and discard the outputs).
- A decoder LSTM is trained to turn the target sequences into
    the same sequence but offset by one timestep in the future,
    a training process called "teacher forcing" in this context.
    It uses as initial state the state vectors from the encoder.
    Effectively, the decoder learns to generate `targets[t+1...]`
    given `targets[...t]`, conditioned on the input sequence.
- In inference mode, when we want to decode unknown input sequences, we:
    - Encode the input sequence into state vectors
    - Start with a target sequence of size 1
        (just the start-of-sequence character)
    - Feed the state vectors and 1-char target sequence
        to the decoder to produce predictions for the next character
    - Sample the next character using these predictions
        (we simply use argmax).
    - Append the sampled character to the target sequence
    - Repeat until we generate the end-of-sequence character or we
        hit the character limit.
# Data download
English to French sentence pairs.
http://www.manythings.org/anki/fra-eng.zip
Lots of neat sentence pairs datasets can be found at:
http://www.manythings.org/anki/
# References
- Sequence to Sequence Learning with Neural Networks
    https://arxiv.org/abs/1409.3215
- Learning Phrase Representations using
    RNN Encoder-Decoder for Statistical Machine Translation
    https://arxiv.org/abs/1406.1078
'''
from __future__ import print_function

from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np

batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.
# Path to the data txt file on disk.
data_path = 'fra-eng/fra.txt'

# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text = line.split('\t')
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype ='float32')

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.

            
            
            

Number of samples: 10000
Number of unique input tokens: 69
Number of unique output tokens: 93
Max sequence length for inputs: 16
Max sequence length for outputs: 59


In [35]:
# updatable plot
# a minimal example (sort of)
import keras
class PlotLosses(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.i = 0
        self.x = []
        self.losses = []
        self.val_losses = []
        self.mse = []
        self.mae = []
        
        self.fig = plt.figure()
        
        self.logs = []

    def on_epoch_end(self, epoch, logs={}):
        
        self.logs.append(logs)
        self.x.append(self.i)
        self.losses.append(logs.get('loss'))
        self.val_losses.append(logs.get('val_loss'))
        self.accuracies.append(logs.get('acc'))
        self.val_accuracies.append(logs.get('val_acc'))
        self.i += 1
        
        clear_output(wait=True)
        plt.plot(self.x, self.losses, label="loss")
        plt.plot(self.x, self.val_losses, label="val_loss")
        plt.plot(self.x, self.accuracies, label="acc")
        plt.plot(self.x, self.val_accuracies, label="val_acc")
        plt.legend()
        plt.show()
        plt.rcParams["figure.figsize"] = [16,9]
        
plot_losses = PlotLosses()

In [36]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense, RNN
layers = [256,128] # we loop LSTMCells then wrap them in an RNN layer

encoder_inputs = keras.layers.Input(shape=(None, num_encoder_tokens))

e_outputs, h1, c1 = LSTM(latent_dim, return_state=True, return_sequences=True)(encoder_inputs) 
_, h2, c2 = LSTM(latent_dim, return_state=True)(e_outputs) 
encoder_states = [h1, c1, h2, c2]

decoder_inputs = Input(shape=(None, num_decoder_tokens))

out_layer1 = LSTM(latent_dim, return_sequences=True, return_state=True)
d_outputs, dh1, dc1 = out_layer1(decoder_inputs,initial_state= [h1, c1])
out_layer2 = LSTM(latent_dim, return_sequences=True, return_state=True)
final, dh2, dc2 = out_layer2(d_outputs, initial_state= [h2, c2])
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(final)


model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_43 (InputLayer)           (None, None, 69)     0                                            
__________________________________________________________________________________________________
input_44 (InputLayer)           (None, None, 93)     0                                            
__________________________________________________________________________________________________
lstm_29 (LSTM)                  [(None, None, 256),  333824      input_43[0][0]                   
__________________________________________________________________________________________________
lstm_31 (LSTM)                  [(None, None, 256),  358400      input_44[0][0]                   
                                                                 lstm_29[0][1]                    
          

In [37]:
epochs = 100
# Run training
model.compile(optimizer='rmsprop', 
              loss='categorical_crossentropy',
              metrics=['acc'])

model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2,
         callbacks=[plot_losses])
# Save model
model.save('s2s.h5')

Train on 8000 samples, validate on 2000 samples
Epoch 1/100
8000/8000 [==============================] - 46s 6ms/step - loss: 0.9346 - acc: 0.0604 - val_loss: 0.9894 - val_acc: 0.0773


AttributeError: 'PlotLosses' object has no attribute 'accuracies'

<Figure size 432x288 with 0 Axes>

In [34]:
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_state_input_h1 = Input(shape=(latent_dim,))
decoder_state_input_c1 = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c, 
                         decoder_state_input_h1, decoder_state_input_c1]
d_o, state_h, state_c = out_layer1(
    decoder_inputs, initial_state=decoder_states_inputs[:2])
d_o, state_h1, state_c1 = out_layer2(
    d_o, initial_state=decoder_states_inputs[-2:])
decoder_states = [state_h, state_c, state_h1, state_c1]
decoder_outputs = decoder_dense(d_o)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

decoder_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_34 (InputLayer)           (None, None, 93)     0                                            
__________________________________________________________________________________________________
input_39 (InputLayer)           (None, 256)          0                                            
__________________________________________________________________________________________________
input_40 (InputLayer)           (None, 256)          0                                            
__________________________________________________________________________________________________
lstm_27 (LSTM)                  [(None, None, 256),  358400      input_34[0][0]                   
                                                                 input_39[0][0]                   
          

In [112]:
# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c, h1, c1 = decoder_model.predict(
            [target_seq] + states_value) #######NOTICE THE ADDITIONAL HIDDEN STATES

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c, h1, c1]#######NOTICE THE ADDITIONAL HIDDEN STATES

    return decoded_sentence


for seq_index in range(100):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Target sentence:', target_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

-
Input sentence: Go.
Target sentence: 	Va !

Decoded sentence: Allez !

-
Input sentence: Hi.
Target sentence: 	Salut !

Decoded sentence: Salut !

-
Input sentence: Run!
Target sentence: 	Cours !

Decoded sentence: Cours !

-
Input sentence: Run!
Target sentence: 	Courez !

Decoded sentence: Cours !

-
Input sentence: Wow!
Target sentence: 	Ça alors !

Decoded sentence: Ça alors !

-
Input sentence: Fire!
Target sentence: 	Au feu !

Decoded sentence: Au ferme !

-
Input sentence: Help!
Target sentence: 	À l'aide !

Decoded sentence: À l'aide !

-
Input sentence: Jump.
Target sentence: 	Saute.

Decoded sentence: Saute.

-
Input sentence: Stop!
Target sentence: 	Ça suffit !

Decoded sentence: Arrête-toi !

-
Input sentence: Stop!
Target sentence: 	Stop !

Decoded sentence: Arrête-toi !

-
Input sentence: Stop!
Target sentence: 	Arrête-toi !

Decoded sentence: Arrête-toi !

-
Input sentence: Wait!
Target sentence: 	Attends !

Decoded sentence: Attends !

-
Input sentence: Wait!
Target s

In [110]:

seq_index = 2
input_seq = encoder_input_data[seq_index: seq_index + 1]
target_seq = np.zeros((1, 1, num_decoder_tokens))
states_value = encoder_model.predict(input_seq)
print('Input sentence:', input_texts[seq_index])
print('Target sentence:', target_texts[seq_index])
input_seq = encoder_input_data[seq_index: seq_index + 1]
states_value = encoder_model.predict(input_seq)
decoded_sentence = ''
stop_condition = False
target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
target_seq[0, 0, target_token_index['\t']] = 1.
target_seq
decoded_sentence = ''


Input sentence: Run!
Target sentence: 	Cours !



In [111]:
output_tokens, h, c, h1, c1 = decoder_model.predict(
            [target_seq] + states_value)
sampled_token_index = np.argmax(output_tokens[0, -1, :])
sampled_char = reverse_target_char_index[sampled_token_index]
if (sampled_char == '\n' or
    len(decoded_sentence) > max_decoder_seq_length):
        stop_condition = True

In [103]:
output_tokens

array([[[3.28468445e-07, 2.67626910e-06, 5.28903753e-02, 3.11445656e-05,
         2.38574557e-06, 7.30980764e-06, 2.00140357e-06, 5.43985261e-05,
         5.32724926e-07, 1.17922002e-06, 8.75312649e-03, 1.54463705e-02,
         2.54699171e-01, 1.52343946e-05, 5.33733055e-06, 1.13827491e-06,
         1.85874012e-06, 2.19590765e-06, 1.19934057e-05, 7.73286138e-06,
         1.29239445e-04, 3.07484479e-05, 3.61205421e-05, 6.93354668e-05,
         8.14259984e-05, 1.44352845e-04, 9.13643598e-05, 1.44697900e-04,
         6.20983483e-05, 7.16865528e-04, 1.00129895e-04, 4.37703238e-06,
         1.96771667e-04, 2.10526676e-04, 7.49556493e-05, 9.77450181e-05,
         1.59461619e-04, 7.86551682e-05, 3.80623853e-04, 9.00865689e-06,
         6.61972490e-06, 3.46562447e-05, 2.70279706e-05, 3.16538535e-05,
         2.43877973e-02, 3.86256637e-04, 2.88435421e-03, 5.91463223e-03,
         3.60161841e-01, 2.01036502e-03, 2.34382343e-03, 8.73518875e-04,
         1.11173624e-02, 1.02480364e-04, 1.09566783

In [104]:
target_seq = np.zeros((1, 1, num_decoder_tokens))
target_seq[0, 0, sampled_token_index] = 1.

        # Update states
states_value = [c, h, c1, h1]
sampled_token_index = np.argmax(output_tokens[0, -1, :])
sampled_char = reverse_target_char_index[sampled_token_index]
decoded_sentence += sampled_char

In [105]:
decoded_sentence

'Aire'